In [41]:
import time
import pandas as pd
import requests

from urllib.parse import quote
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

- 네이버 지식iN : 어버이날 선물로

In [20]:
sword = quote('어버이날 선물')
nurl = 'https://kin.naver.com/search/list.naver?query=' + sword
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe')

In [21]:
driver.get(nurl)
time.sleep(2)

- 전체 개수 구하기

In [31]:
int(driver.find_element(By.CLASS_NAME, 'number').text.strip('()').split('/')[-1].replace(',',''))

83825

In [12]:
urls = driver.find_elements(By.XPATH, '//*[@id="s_content"]/div[3]/ul/li')
len(urls)

10

In [13]:
urls[0].text

'40대 50대 어버이날선물 추천 해주세요 2023.04.25.\n곧 어버이날이여서 미리 준비할려구용~ 어버이날선물추천 해주세요!!! 부모님 두분다 만족하실 수 있는 선물로 부탁드려용 오빠랑 합쳐서 50만원정도 있습니당 50대...\nQ&A > 가족행사, 이벤트 | 답변수 1  추천수 0 | 답변 종로금빵(whdf****)'

In [14]:
url_list = []
for url in urls:
    href = url.find_element(By.TAG_NAME, 'a')
    url_list.append(href.get_attribute('href'))    

- 2page로 이동

In [15]:
driver.find_element(By.XPATH, '//*[@id="s_content"]/div[3]/div[2]/a[1]').click()
time.sleep(2)

In [16]:
urls = driver.find_elements(By.XPATH, '//*[@id="s_content"]/div[3]/ul/li')
len(urls)

10

In [17]:
for url in urls:
    href = url.find_element(By.TAG_NAME, 'a')
    url_list.append(href.get_attribute('href'))   

In [18]:
len(url_list)

20

In [32]:
driver.close()

- 반복해서 가져와 보기

In [37]:
def get_total_page(cnt):
    if cnt > 100:
        return 10
    else:
        return int(cnt / 10)


In [57]:

driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe')
driver.get(nurl)
time.sleep(2)

urls = driver.find_elements(By.XPATH, '//*[@id="s_content"]/div[3]/ul/li')

# 전체 지식in 조회 수
cnt = int(driver.find_element(By.CLASS_NAME, 'number').text.strip('()').split('/')[-1].replace(',',''))
total_page = get_total_page(cnt)
print(total_page)

url_list = []
npage = 1
while True:    
    for url in urls:

        href = url.find_element(By.TAG_NAME, 'dl')
        hit = int(href.find_element(By.CLASS_NAME, 'hit').text.split()[-1].strip())
       
        
        href = href.find_element(By.TAG_NAME, 'a')
        title = href.text
        uurl = href.get_attribute('href')
        
        url_list.append({'제목':title, 'URL': uurl, '답변수':hit})

    if npage >= total_page:
        break

    driver.find_element(By.XPATH, f'//*[@id="s_content"]/div[3]/div[2]/a[{npage}]').click()
    time.sleep(2)

    urls = driver.find_elements(By.XPATH, '//*[@id="s_content"]/div[3]/ul/li')
    npage += 1

driver.close()

df = pd.DataFrame(url_list)
df.head()


10


,제목,URL,답변수
0,40대 50대 어버이날선물 추천 해주세요,https://kin.naver.com/qna/detail.naver?d1id=8&...,1
1,어버이날 선물 뭐가 좋을까요,https://kin.naver.com/qna/detail.naver?d1id=8&...,2
2,어버이날 선물 추천 부탁해요~,https://kin.naver.com/qna/detail.naver?d1id=3&...,2
3,어버이날 골프 관련 선물,https://kin.naver.com/qna/detail.naver?d1id=10...,7
4,40대 아버지 어버이날 선물,https://kin.naver.com/qna/detail.naver?d1id=8&...,19


In [39]:
print(len(url_list))

100


- sub 답변들 가져오기

In [40]:
for url in url_list[4]:
    driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe')
    driver.get(nurl)
    time.sleep(2)


NameError: name 'requests' is not defined